In [72]:
from contextlib import ExitStack

from dms_stan.utils import from_csv_noload

import numpy as np
import dask.array as da
import h5py

In [2]:
test = from_csv_noload(
    "/home/bwittmann//GitRepos/DMSStan/flip3/trpB/exponential/libI-20250429183759_*.csv"
)

In [29]:
test.runset.args

AttributeError: 'RunSet' object has no attribute 'args'

In [47]:
# Get a copy of the config object
config = test.metadata.cmdstan_config

# Get the number of draws and number of warmups
num_draws = test.num_draws_sampling
sampling_iter_start = 0
if config["save_warmup"]:
    num_draws += test.num_draws_warmup
    sampling_iter_start = test.num_draws_warmup

# # Build empty dask arrays for each of the variables
# draws = {
#     varname: da.empty(
#         dtype=np.float32,
#         shape=()
#     )
# }
# test.num_draws_sampling

In [ ]:
# TODO: Writing to hdf5 files should be the standard way of processing outputs of
# the cmdstan sampling run. This will make it very easy for us to toggle the dask
# backend off and on

method_var_dtypes = {
    "lp__": np.float32,
    "accept_stat__": np.float32,
    "stepsize__": np.float32,
    "treedepth__": np.int32,
    "n_leapfrog__": np.int32,
    "divergent__": np.bool_,
    "energy__": np.float32,
}

# Build the hdf5 file
with h5py.File("/home/bwittmann/GitRepos/DMSStan/tmp/test.hdf5", "w") as f:

    # We need a group for metadata, samples, and posterior predictive checks
    metadata_group = f.create_group("metadata")
    sample_group = f.create_group("samples")
    ppc_group = f.create_group("ppc")

    # Create datasets for each variable of the metadata group
    metadata_dsets = {
        varname: metadata_group.create_dataset(
            name=varname,
            shape=(config["num_chains"], num_draws),
            dtype=method_var_dtypes[varname],
            chunks=(1, num_draws),  # TODO: make this configurable
        )
        for varname in test.metadata.method_vars.keys()
    }

    # Create datasets for each variable of the sample group, including posterior
    # predictive checks
    sample_dsets, ppc_dsets = {}, {}
    for varname, varspec in test.metadata.stan_vars.items():

        # Determine the dictionary to use
        target = ppc_dsets if varname.endswith("_ppc") else sample_dsets

        # Build the dataset
        target[varname] = ppc_group.create_dataset(
            name=varname,
            shape=(config["num_chains"], num_draws, *varspec.dimensions),
            dtype=np.float32,  # TODO: inherit from the model
            chunks=(1, num_draws, *varspec.dimensions),  # TODO: make this configurable
        )

    # Parse each of the csv files
    for chain_ind, csv_file in enumerate(sorted(test.runset.csv_files)):
        for draw_ind, draw_data in enumerate(parse_csv(csv_file)):
            # TODO: We need to write the `parse_csv` generator. This should emit
            # a dictionary of the form: varname -> array of sampled data. These
            # arrays will then be written to the appropriate dataset.
            pass

In [ ]:
test.runset.it

[<_io.TextIOWrapper name='/home/bwittmann//GitRepos/DMSStan/flip3/trpB/exponential/libI-20250429183759_1.csv' mode='r' encoding='utf-8'>,
 <_io.TextIOWrapper name='/home/bwittmann//GitRepos/DMSStan/flip3/trpB/exponential/libI-20250429183759_2.csv' mode='r' encoding='utf-8'>,
 <_io.TextIOWrapper name='/home/bwittmann//GitRepos/DMSStan/flip3/trpB/exponential/libI-20250429183759_3.csv' mode='r' encoding='utf-8'>,
 <_io.TextIOWrapper name='/home/bwittmann//GitRepos/DMSStan/flip3/trpB/exponential/libI-20250429183759_4.csv' mode='r' encoding='utf-8'>]

In [60]:
test.metadata.stan_vars["inv_r_mean"].dimensions

(9121,)

In [39]:
bool("0")

True